# Querying Data Lakes with DataFrames

Apache Spark&trade; and Azure Databricks&reg; make it easy to access and work with files stored in Data Lakes, such as Azure Data Lake Storage (ADLS).

Companies frequently store thousands of large data files gathered from various teams and departments, typically using a diverse variety of formats including CSV, JSON, and XML. Data scientists often wish to extract insights from this data.

The classic approach to querying this data is to load it into a central database called a **data warehouse**. Traditionally, data engineers must design the schema for the central database, extract the data from the various data sources, transform the data to fit the warehouse schema, and load it into the central database. A data scientist can then query the data warehouse directly or query smaller data sets created to optimize specific types of queries. The data warehouse approach works well, but requires a great deal of up front effort to design and populate schemas. It also limits historical data, which is constrained to only the data that fits the warehouse’s schema.

An alternative approach is a **Data Lake**, which:

* Is a storage repository that cheaply stores a vast amount of raw data in its native format.
* Consists of current and historical data dumps in various formats including XML, JSON, CSV, Parquet, etc.
* May contain operational relational databases with live transactional data.

Spark is ideal for querying Data Lakes. Spark DataFrames can be used to read directly from raw files contained in a Data Lake and then execute queries to join and aggregate the data.

This lesson illustrates how to perform exploratory data analysis (EDA) to gain insights from a Data Lake.

## Prerequisites
* **IMPORTANT**: You must have permissions within your Azure subscription to create an App Registration and service principal within Azure Active Directory to complete this lesson.
* Lesson: <a href="$./02-Querying-Files">Querying Files with SQL</a>

### Getting Started

Run the following cell to configure our "classroom."

In [3]:
%run "./Includes/Classroom-Setup"

## Create Azure Data Lake Storage Gen1 (ADLS)

1. In the [Azure portal](https://portal.azure.com), select **+ Create a resource**, enter "data lake" into the Search the Marketplace box, select **Data Lake Storage Gen1** from the results, and then select **Create**.

   ![In the Azure portal, +Create a resource is highlighted in the navigation pane, "data lake" is entered into the Search the Marketplace box, and Data Lake Storage Gen1 is highlighted in the results.](https://databricksdemostore.blob.core.windows.net/images/04/06/adls-create-resource.png 'Create Azure Data Lake Storage Gen1')

2. On the New Data Lake Storage Gen1 blade, enter the following:

   - **Name**: Enter a globally unique name (indicated by a green check mark).
   - **Subscription**: Select the subscription you are using for this module.
   - **Resource group**: Choose your module resource group.
   - **Location**: Select the closest location.
   - **Pricing package**: Choose Pay-as-You-Go.
   - **Encryption settings**: Leave set to the default value of Enabled.

   ![The New Data Lake Storage Gen1 blade is displayed, with the previously mentioned settings entered into the appropriate fields.](https://databricksdemostore.blob.core.windows.net/images/04/06/adls-create-new.png 'New Data Lake Storage Gen1')

3. Select **Create** to provision the new ADLS instance.

4. In the cell below, set the value of the `adlsAccountName` variable to the same name you used for the **Name** field when creating your ADLS instance above, and then run the cell.

In [5]:
adlsAccountName = "warrentestgen1"

## Create Azure Active Directory application and service principal

> **IMPORTANT**: You must have permissions within your Azure subscription to create an App registration and service principal within Azure Active Directory to complete this lesson.

ADLS uses Azure Active Directory for authentication. To provide access to your ADLS instance from Azure Databricks, you will use [service-to-service authentication](https://docs.microsoft.com/en-us/azure/data-lake-store/data-lake-store-service-to-service-authenticate-using-active-directory). For this, you need to create an identity in Azure Active Directory (Azure AD) known as a service principal.

1. In the [Azure portal](https://portal.azure.com), select **Azure Active Directory** from the left-hand navigation menu, select **App registrations**, and then select **+ New application registration**.

   ![Register new app in Azure Active Directory](https://databricksdemostore.blob.core.windows.net/images/04/06/aad-app-registration.png 'Register new app in Azure Active Directory')

2. On the Create blade, enter the following:

  * **Name**: Enter a unique name, such as databricks-demo (this name must be unique, as indicated by a green check mark).
  * **Application type**: Select Web app / API.
  * **Sign-on URL**: Enter https://databricks-demo.com.

   ![Create a new app registration](https://databricksdemostore.blob.core.windows.net/images/04/06/aad-app-create.png 'Create a new app registration')

3. Select **Create**.

4. To access your ADLS instance from Azure Databricks you will need to provide the credentials of your newly created service principal within Databricks. On the Registered app blade that appears, copy the **Application ID** and paste it into the cell below as the value for the `clientId` variable.

   ![Copy the Registered App Application ID](https://databricksdemostore.blob.core.windows.net/images/04/06/registered-app-id.png 'Copy the Registered App Application ID')

5. Next, select **Settings** on the Registered app blade, and then select **Keys**.

   ![Open Keys blade for the Registered App](https://databricksdemostore.blob.core.windows.net/images/04/06/registered-app-settings-keys.png 'Open Keys blade for the Registered App')

6. On the Keys blade, you will create a new password by doing the following under Passwords:

  * **Description**: Enter a description, such as ADLS Auth.
  * **Expires**: Select a duration, such as In 1 year.

  ![Create new password](https://databricksdemostore.blob.core.windows.net/images/04/06/registered-app-create-key.png 'Create new password')

7. Select **Save**, and then copy the key displayed under **Value**, and paste it into the cell below for the value of the `clientKey` variable. **Note**: This value will not be accessible once you navigate away from this screen, so make sure you copy it before leaving the Keys blade.

  ![Copy key value](https://databricksdemostore.blob.core.windows.net/images/04/06/registered-app-key-value.png 'Copy key value')

8. Run the cell below.

In [7]:
clientId = "9a14adb0-d7a6-4021-80f3-f47115fc1597"
clientKey = "F21cE5gcgIlP.y8ZCH-hSmt]hEGug[fn"

## Retrieve your Azure AD tenant ID

To perform authentication using the service principal account, Databricks uses OAUTH2. For this, you need to provide your Azure AD Tenant ID.

1. To retrieve your tenant ID, select **Azure Active Directory** from the left-hand navigation menu in the Azure portal, then select **Properties**, and select the copy button next to **Directory ID** on the Directory Properties blade.

   ![Retrieve Tenant ID](https://databricksdemostore.blob.core.windows.net/images/04/06/aad-tenant-id.png 'Retrieve Tenant ID')

2. Paste the copied value into the cell below for the value of the `tenantId` variable, and then run the cell.

In [9]:
tenantId = "1a5d5bac-c4df-4a2c-9374-267fcff8eead"

## Assign permissions to the service principal in ADLS

Next, you need to assign the required permissions to the service principal in ADLS.

1. In the [Azure portal](https://portal.azure.com), navigate to the ADLS instance you created above, and on the Overview blade, select **Data explorer**.

   ![ADLS Overview blade](https://databricksdemostore.blob.core.windows.net/images/04/06/adls-overview.png 'ADLS Overview blade')

2. In the Data Explorer blade, select **Access** on the toolbar.

   ![ADLS Data Explorer toolbar](https://databricksdemostore.blob.core.windows.net/images/04/06/adls-data-explorer-toolbar.png 'ADLS Data Explorer toolbar')

3. On the Access blade, select **+ Add**.

   ![ADLS Data Explorer add access](https://databricksdemostore.blob.core.windows.net/images/04/06/adls-access.png 'ADLS Data Explorer add access')

4. On the Assign permissions -> Select user or group blade, enter the name of your Registered app (e.g., databricks-demo) into the **Select** box, choose your app from the list, and select **Select**.

   ![ADLS assign permissions to user or group](https://databricksdemostore.blob.core.windows.net/images/04/06/adls-assign-permissions-select-user-or-group.png 'ADLS assign permissions to user or group')

5. On the Assign permissions -> Select permissions blade, set the following:

  * **Permissions**: Check **Read**, **Write**, and **Execute**.
  * **Add to**: Choose This folder and all children.
  * **Add as**: Choose An access permission entry.
  
  ![ADLS assign permissions](https://databricksdemostore.blob.core.windows.net/images/04/06/adls-assign-permissions.png 'ADLS assign permissions')

6. Select **Ok**

7. You will now see the service principal listed under **Assigned permissions** on the Access blade.

  ![ADLS assigned permissions](https://databricksdemostore.blob.core.windows.net/images/04/06/adls-assigned-permissions.png 'ADLS assigned permissions')

## Mount ADLS to DBFS

You are now ready to access your ADLS account from Azure Databricks. Run the cell below to set the required configuration and mount ADLS to DBFS.

In [12]:
configs = {"dfs.adls.oauth2.access.token.provider.type": "ClientCredential",
           "dfs.adls.oauth2.client.id": clientId,
           "dfs.adls.oauth2.credential": clientKey,
           "dfs.adls.oauth2.refresh.url": "https://login.microsoftonline.com/" + tenantId + "/oauth2/token"}

dbutils.fs.mount(
  source = "adl://warrentestgen1.azuredatalakestore.net/",
  mount_point = "/mnt/adls",
  extra_configs = configs)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-4180736491747904> in <module> 
 7 source = "adl://warrentestgen1.azuredatalakestore.net/" , 
 8 mount_point = "/mnt/adls" , 
 ----> 9 extra_configs = configs)
 
 /local_disk0/tmp/1588856738755-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 312 exc . __context__ = None 
 313 exc . __cause__ = None 
 --> 314 raise exc
 315 return f_with_exception_handling
 316 

 ExecutionError : An error occurred while calling o323.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/adls; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/adls
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:63)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:465)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/adls
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:224)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:326)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:220)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:79)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:103)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:102)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:102)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:304)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:282)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:51)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:78)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:78)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:47)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:428)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:230)
	at com.databricks.rpc.ServerBackend.withAttributionContext(ServerBackend.scala:14)
	at com.databric

## Copy data to ADLS

Run the following cell to copy the Crime-data-2016 dataset from the Training folder into your ADLS instance, in a folder named "training". This will take a few minutes to complete.

In [14]:
dbutils.fs.cp("/mnt/training/crime-data-2016", "mnt/adls/training/crime-data-2016", True)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-4180736491747906> in <module> 
 ----> 1 dbutils . fs . cp ( "/mnt/training/crime-data-2016" , "mnt/adls/training/crime-data-2016" , True ) 

 /local_disk0/tmp/1588708292664-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 312 exc . __context__ = None 
 313 exc . __cause__ = None 
 --> 314 raise exc
 315 return f_with_exception_handling
 316 

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.cp.
: com.microsoft.azure.datalake.store.ADLException: Error creating directory /training/crime-data-2016
Operation MKDIRS failed with exception java.net.UnknownHostException : warretest.azuredatalakestore.net
Last encountered exception thrown after 5 tries. [java.net.UnknownHostException,java.net.UnknownHostException,java.net.UnknownHostException,java.net.UnknownHostException,java.net.UnknownHostException]
 [ServerRequestId:null]
	at com.microsoft.azure.datalake.store.ADLStoreClient.getExceptionFromResponse(ADLStoreClient.java:1169)
	at com.microsoft.azure.datalake.store.ADLStoreClient.createDirectory(ADLStoreClient.java:589)
	at com.databricks.adl.AdlFileSystem.mkdirs(AdlFileSystem.java:581)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$mkdirs$1$$anonfun$apply$mcZ$sp$7$$anonfun$apply$mcZ$sp$8.apply$mcZ$sp(DatabricksFileSystemV2.scala:743)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$mkdirs$1$$anonfun$apply$mcZ$sp$7$$anonfun$apply$mcZ$sp$8.apply(DatabricksFileSystemV2.scala:741)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$mkdirs$1$$anonfun$apply$mcZ$sp$7$$anonfun$apply$mcZ$sp$8.apply(DatabricksFileSystemV2.scala:741)
	at com.databricks.s3a.S3AExeceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:119)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$mkdirs$1$$anonfun$apply$mcZ$sp$7.apply$mcZ$sp(DatabricksFileSystemV2.scala:741)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$mkdirs$1$$anonfun$apply$mcZ$sp$7.apply(DatabricksFileSystemV2.scala:741)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$mkdirs$1$$anonfun$apply$mcZ$sp$7.apply(DatabricksFileSystemV2.scala:741)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$withUserContextRecorded$1.apply(DatabricksFileSystemV2.scala:936)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:251)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:246)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:450)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:288)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:450)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withUserContextRecorded(DatabricksFileSystemV2.scala:909)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$mkdirs$1.apply$mcZ$sp(DatabricksFileSystemV2.scala:740)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$mkdirs$1.apply(DatabricksFileSystemV2.scala:740)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$mkdirs$1.apply(DatabricksFileSystemV2.scala:740)
	at com.databricks.logging.UsageLogging$$anonfun$recordOperation$1.apply(UsageLogging.scala:440)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:251)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(

## Looking at the files in our Data Lake

Start by reviewing which files are in our Data Lake.

In `dbfs:/mnt/adls/training/crime-data-2016`, there are Parquet files containing 2016 crime data from several United States cities.

In the cell below we have data for Boston, Chicago, New Orleans, and more.

In [16]:
%fs ls /mnt/adls/training/crime-data-2016

Operation LISTSTATUS failed with exception java.net.UnknownHostException : warretest.azuredatalakestore.net
Last encountered exception thrown after 5 tries. [java.net.UnknownHostException,java.net.UnknownHostException,java.net.UnknownHostException,java.net.UnknownHostException,java.net.UnknownHostException]
 [ServerRequestId:null]
	at com.microsoft.azure.datalake.store.ADLStoreClient.getExceptionFromResponse(ADLStoreClient.java:1169)
	at com.microsoft.azure.datalake.store.ADLStoreClient.enumerateDirectoryInternal(ADLStoreClient.java:558)
	at com.microsoft.azure.datalake.store.ADLStoreClient.enumerateDirectory(ADLStoreClient.java:534)
	at com.microsoft.azure.datalake.store.ADLStoreClient.enumerateDirectory(ADLStoreClient.java:398)
	at com.microsoft.azure.datalake.store.ADLStoreClient.enumerateDirectory(ADLStoreClient.java:384)
	at com.databricks.adl.AdlFileSystem.listStatus(AdlFileSystem.java:463)
	at com.databricks.backend.daemon.data.client.DBFSV2$$anonfun$listStatus$1$$anonfun$apply$2.apply(DatabricksFileSystemV2.scala:95)
	at com.databricks.backend.daemon.data.client.DBFSV2$$anonfun$listStatus$1$$anonfun$apply$2.apply(DatabricksFileSystemV2.scala:92)
	at com.databricks.s3a.S3AExeceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:119)
	at com.databricks.backend.daemon.data.client.DBFSV2$$anonfun$listStatus$1.apply(DatabricksFileSystemV2.scala:92)
	at com.databricks.backend.daemon.data.client.DBFSV2$$anonfun$listStatus$1.apply(DatabricksFileSystemV2.scala:92)
	at com.databricks.logging.UsageLogging$$anonfun$recordOperation$1.apply(UsageLogging.scala:440)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:251)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:246)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:450)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:288)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:450)
	at com.databricks.logging.UsageLogging$class.recordOperation(UsageLogging.scala:421)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperation(DatabricksFileSystemV2.scala:450)
	at com.databricks.backend.daemon.data.client.DBFSV2.listStatus(DatabricksFileSystemV2.scala:91)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala:150)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$ls$1.apply(DBUtilsCore.scala:86)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$ls$1.apply(DBUtilsCore.scala:85)
	at com.databricks.backend.daemon.dbutils.FSUtils$.com$databricks$backend$daemon$dbutils$FSUtils$$withFsSafetyCheck(DBUtilsCore.scala:81)
	at com.databricks.backend.daemon.dbutils.FSUtils$.ls(DBUtilsCore.scala:85)
	at com.databricks.dbutils_v1.impl.DbfsUtilsImpl.ls(DbfsUtilsImpl.scala:34)
	at line1420c957e3cc45cd871ddb783dd97aa233.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-4180736491747908:1)
	at line1420c957e3cc45cd871ddb783dd97aa233.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-4180736491747908:44)
	at line1420c957e3cc45cd871ddb783dd97aa233.$read$$iw$$iw$$iw$$iw.<init>(command-4180736491747908:46)
	at line1420c957e3cc45cd871ddb783dd97aa233.$read$$iw$$iw$$iw.<init>(command-4180736491747908:48)
	at line1420c957e3cc45cd871ddb783dd97aa233.$read$$iw$$iw.<init>(command-4180736491747908:50)
	at line1420c957e3cc45cd871ddb783dd97aa233.$read$$iw.<init>(command-4180736491747908:52)
	at line1420c957e3cc45cd871ddb783dd97aa233.$read.<init>(command-4180736491747908:54)
	at line1420c957e3cc45cd871ddb783dd97aa233.$read$.<init>(command-4180736491747908:58)
	at line1420c957e3cc45cd871ddb783dd97aa233.$read$.<clinit>(command-4180736491747908)
	at line1420c957e3cc45cd871ddb783dd97aa233.$eval$.$p

The next step in looking at the data is to create a temporary view for each file.  Recall that temporary views use a similar syntax to `CREATE TABLE` but using the command `CREATE TEMPORARY VIEW`.  Temporary views are removed once your session has ended while tables are persisted beyond a given session.

Start by creating a view of the data from New York and then Boston:

| City          | Table Name              | Path to DBFS file
| ------------- | ----------------------- | -----------------
| **New York**  | `CrimeDataNewYork`      | `dbfs:/mnt/adls/training/crime-data-2016/Crime-Data-New-York-2016.parquet`
| **Boston**    | `CrimeDataBoston`       | `dbfs:/mnt/adls/training/crime-data-2016/Crime-Data-Boston-2016.parquet`

In [18]:
%sql

CREATE OR REPLACE TEMPORARY VIEW CrimeDataNewYork
  USING parquet
  OPTIONS (
    path "dbfs:/mnt/adls/training/crime-data-2016/Crime-Data-New-York-2016.parquet"
  )

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: com.microsoft.azure.datalake.store.ADLException: Error getting info for file /training/crime-data-2016/Crime-Data-New-York-2016.parquet
Operation GETFILESTATUS failed with exception java.net.UnknownHostException : warretest.azuredatalakestore.net
Last encountered exception thrown after 5 tries. [java.net.UnknownHostException,java.net.UnknownHostException,java.net.UnknownHostException,java.net.UnknownHostException,java.net.UnknownHostException]
 [ServerRequestId:null]
	at com.microsoft.azure.datalake.store.ADLStoreClient.getExceptionFromResponse(ADLStoreClient.java:1169)
	at com.microsoft.azure.datalake.store.ADLStoreClient.getDirectoryEntry(ADLStoreClient.java:737)
	at com.microsoft.azure.datalake.store.ADLStoreClient.getDirectoryEntry(ADLStoreClient.java:718)
	at com.databricks.adl.AdlFileSystem.getFileStatus(AdlFileSystem.java:446)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$getFileStatus$1$$anonfun$apply$15.apply(DatabricksFileSystemV2.scala:759)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$getFileStatus$1$$anonfun$apply$15.apply(DatabricksFileSystemV2.scala:756)
	at com.databricks.s3a.S3AExeceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:119)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$getFileStatus$1.apply(DatabricksFileSystemV2.scala:756)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$getFileStatus$1.apply(DatabricksFileSystemV2.scala:756)
	at com.databricks.logging.UsageLogging$$anonfun$recordOperation$1.apply(UsageLogging.scala:440)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:251)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:246)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:450)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:288)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:450)
	at com.databricks.logging.UsageLogging$class.recordOperation(UsageLogging.scala:421)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperation(DatabricksFileSystemV2.scala:450)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.getFileStatus(DatabricksFileSystemV2.scala:755)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.getFileStatus(DatabricksFileSystem.scala:201)
	at org.apache.hadoop.fs.FileSystem.exists(FileSystem.java:1426)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:613)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:597)
	at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
	at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike$class.flatMap(TraversableLike.scala:241)
	at scala.collection.immutable.List.flatMap(List.scala:355)
	at org.apache.spark.sql.execution.datasources.DataSource.org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary(DataSource.scala:597)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:391)
	at org.apache.spark.sql.execution.datasources.CreateTempViewUsing.run(ddl.scala:101)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)


In [19]:
%sql

CREATE OR REPLACE TEMPORARY VIEW CrimeDataBoston
  USING parquet
  OPTIONS (
    path "dbfs:/mnt/adls/training/crime-data-2016/Crime-Data-Boston-2016.parquet"
  )

With the view created, it is now possible to review the first couple records of each file.

Notice in the example below:
* The `CrimeDataNewYork` and `CrimeDataBoston` datasets use different names for the columns
* The data itself is formatted differently and different names are used for similar concepts

This is common in a Data Lake.  Often files are added to a Data Lake by different groups at different times.  While each file itself usually has clean data, there is little consistency across files.  The advantage of this strategy is that anyone can contribute information to the Data Lake and that Data Lakes scale to store arbitrarily large and diverse data.  The tradeoff for this ease in storing data is that it doesn’t have the rigid structure of a more traditional relational data model so the person querying the Data Lake will need to clean the data before extracting useful insights.

The alternative to a Data Lake is a Data Warehouse.  In a Data Warehouse, a committee often regulates the schema and ensures data is cleaned before being made available.  This makes querying much easier but also makes gathering the data much more expensive and time-consuming.  Many companies choose to start with a Data Lake to accumulate data.  Then, as the need arises, they clean the data and produce higher quality tables for querying.  This reduces the upfront costs while still making data easier to query over time.  These cleaned tables can even be later loaded into a formal data warehouse through nightly batch jobs.  In this way, Apache Spark can be used to manage and query both Data Lakes and Data Warehouses.

In [21]:
%sql

SELECT * FROM CrimeDataNewYork

In [22]:
%sql

SELECT * FROM CrimeDataBoston

## Same type of data, different structure

In this section, we examine crime data to figure out how to extract homicide statistics.

Because our data sets are pooled together in a Data Lake, each city may use different field names and values to indicate homicides, dates, etc.

For example:
* Some cities use the value "HOMICIDE", "CRIMINAL HOMICIDE" or even "MURDER"
* In New York, the column is named `offenseDescription` but, in Boston, the column is named `OFFENSE_CODE_GROUP`
* In New York, the date of the event is in the `reportDate` column but, in Boston, there is a single column named `MONTH`

-sandbox

To get started, create a temporary view containing only the homicide-related rows.

At the same time, normalize the data structure of each table so that all the columns (and their values) line up with each other.

In the case of New York and Boston, here are the unique characteristics of each data set:

| | Offense-Column        | Offense-Value          | Reported-Column  | Reported-Data Type |
|-|-----------------------|------------------------|-----------------------------------|
| New York | `offenseDescription`  | starts with "murder" or "homicide" | `reportDate`     | `timestamp`    |
| Boston | `OFFENSE_CODE_GROUP`  | "Homicide"             | `MONTH`          | `integer`      |

For the upcoming aggregation, you will need to alter the New York data set to include a `month` column which can be computed from the `reportDate` column using the `month()` function. Boston already has this column.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> One helpful tool for finding the offences we're looking for is using <a href="https://en.wikipedia.org/wiki/Regular_expression" target="_blank">regular expressions</a> supported by SQL

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> We can also normalize the values with the `CASE`, `WHEN`, `THEN` & `ELSE` expressions but that is not required for the task at hand.

In [25]:
%sql

CREATE OR REPLACE TEMPORARY VIEW HomicidesNewYork AS
  SELECT month(reportDate) AS month, offenseDescription AS offense
  FROM CrimeDataNewYork
  WHERE lower(offenseDescription) LIKE 'murder%' OR lower(offenseDescription) LIKE 'homicide%'

In [26]:
%sql

CREATE OR REPLACE TEMPORARY VIEW HomicidesBoston AS
  SELECT month, OFFENSE_CODE_GROUP AS offense
  FROM CrimeDataBoston
  WHERE lower(OFFENSE_CODE_GROUP) = 'homicide'

You can see below that the structure of our two tables is now identical.

In [28]:
%sql

SELECT * FROM HomicidesNewYork LIMIT 5

In [29]:
%sql

SELECT * FROM HomicidesBoston LIMIT 5

## Analyzing the data

-sandbox
Now that we have normalized the homicide data for each city we can combine the two by taking their union.

When we are done, we can then aggregate that data to compute the number of homicides per month.

Start by creating a new view called `HomicidesBostonAndNewYork` which simply unions the result of two `SELECT` statements together.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> See <a href="https://stackoverflow.com/questions/49925/what-is-the-difference-between-union-and-union-all">this Stack Overflow post</a> for the difference between `UNION` and `UNION ALL`

In [32]:
%sql

CREATE OR REPLACE TEMPORARY VIEW HomicidesBostonAndNewYork AS
  SELECT * FROM HomicidesNewYork
    UNION ALL
  SELECT * FROM HomicidesBoston

You can now see below all the data in one table:

In [34]:
%sql

SELECT *
FROM HomicidesBostonAndNewYork
ORDER BY month

And finally we can perform a simple aggregation to see the number of homicides per month:

In [36]:
%sql

SELECT month, count(*) AS homicides
FROM HomicidesBostonAndNewYork
GROUP BY month
ORDER BY month

## Exercise 1

Merge the crime data for Chicago with the data for New York and Boston and then update our final aggregation of counts-by-month.

### Step 1

Create the initial view of the Chicago data.
0. The source file is `dbfs:/mnt/adls/training/crime-data-2016/Crime-Data-Chicago-2016.parquet`
0. Name the view `CrimeDataChicago`
0. View the data with a simple `SELECT` statement

In [39]:
%sql
-- TODO

<<FILL_IN>>

In [40]:
%python
# TEST - Run this cell to test your solution.

total = spark.sql("select count(*) from CrimeDataChicago").first()[0]
dbTest("SQL-L6-crimeDataChicago-count", 267872, total)

print("Tests passed!")

-sandbox
### Step 2

Create a new view that normalizes the data structure.
0. Name the view `HomicidesChicago`
0. The table should have at least two columns: `month` and `offense`
0. Filter the data to only include homicides
0. View the data with a simple `SELECT` statement

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** You will need to use the `month()` function to extract the month-of-the-year.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** To find out which values for each offense constitutes a homicide, produce a distinct list of values from the table `CrimeDataChicago`.

In [42]:
%sql
-- TODO

<<FILL_IN>>

In [43]:
%python
# TEST - Run this cell to test your solution.

homicidesChicago = spark.sql("SELECT month, count(*) FROM HomicidesChicago GROUP BY month ORDER BY month").collect()
dbTest("SQL-L6-homicideChicago-len", 12, len(homicidesChicago))

dbTest("SQL-L6-homicideChicago-0", 54, homicidesChicago[0][1])
dbTest("SQL-L6-homicideChicago-6", 71, homicidesChicago[6][1])
dbTest("SQL-L6-homicideChicago-11", 58, homicidesChicago[11][1])

print("Tests passed!")

-sandbox
### Step 3

Create a new view that merges all three data sets (New York, Boston, Chicago):
0. Name the view `AllHomicides`
0. Use the `UNION ALL` expression introduced earlier to merge all three tables
  * `HomicidesNewYork`
  * `HomicidesBoston`
  * `HomicidesChicago`
0. View the data with a simple `SELECT` statement

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** To union three tables together, copy the previous example and just add as second `UNION` statement followed by the appropriate `SELECT` statement.

In [45]:
%sql
-- TODO

<<FILL_IN>>

In [46]:
%python
# TEST - Run this cell to test your solution.

allHomicides = spark.sql("SELECT count(*) AS total FROM AllHomicides").first().total
dbTest("SQL-L6-allHomicides-count", 1203, allHomicides)

print("Tests passed!")

### Step 4

Create a new view that counts the number of homicides per month.
0. Name the view `HomicidesByMonth`
0. Rename the column `count(1)` to `homicides`
0. Group the data by `month`
0. Sort the data by `month`
0. Count the number of records for each aggregate
0. View the data with a simple `SELECT` statement

In [48]:
%sql
-- TODO

<<FILL_IN>>

In [49]:
%python
# TEST - Run this cell to test your solution.

allHomicides = spark.sql("SELECT * FROM HomicidesByMonth").collect()
dbTest("SQL-L6-homicidesByMonth-len", 12, len(allHomicides))

dbTest("SQL-L6-homicidesByMonth-0", 1, allHomicides[0].month)
dbTest("SQL-L6-homicidesByMonth-11", 12, allHomicides[11].month)

dbTest("SQL-L6-allHomicides-0", 83, allHomicides[0].homicides)
dbTest("SQL-L6-allHomicides-1", 68, allHomicides[1].homicides)
dbTest("SQL-L6-allHomicides-2", 72, allHomicides[2].homicides)
dbTest("SQL-L6-allHomicides-3", 76, allHomicides[3].homicides)
dbTest("SQL-L6-allHomicides-4", 105, allHomicides[4].homicides)
dbTest("SQL-L6-allHomicides-5", 120, allHomicides[5].homicides)
dbTest("SQL-L6-allHomicides-6", 116, allHomicides[6].homicides)
dbTest("SQL-L6-allHomicides-7", 144, allHomicides[7].homicides)
dbTest("SQL-L6-allHomicides-8", 109, allHomicides[8].homicides)
dbTest("SQL-L6-allHomicides-9", 109, allHomicides[9].homicides)
dbTest("SQL-L6-allHomicides-10", 111, allHomicides[10].homicides)
dbTest("SQL-L6-allHomicides-11", 90, allHomicides[11].homicides)

print("Tests passed!")

## Unmount ADLS from DBFS

In [51]:
dbutils.fs.unmount("/mnt/adls")

## Summary

* Spark SQL allows you to easily manipulate data in a Data Lake
* Temporary views help to save your cleaned data for downstream analysis

## Review Questions
**Q:** What is a Data Lake?  
**A:** Data Lakes are a loose collection of data files gathered from various sources.  Spark loads each file as a table and then executes queries joining and aggregating these files.

**Q:** What are some advantages of Data Lakes over more classic Data Warehouses?  
**A:** Data Lakes allow for large amounts of data to be aggregated from many sources with minimal ceremony or overhead.  Data Lakes also allow for very very large files.  Powerful query engines such as Spark can read the diverse collection of files and execute complex queries efficiently.

**Q:** What are some advantages of Data Warehouses?  
**A:** Data warehouses are neatly curated to ensure data from all sources fit a common schema.  This makes them very easy to query.

**Q:** What's the best way to combine the advantages of Data Lakes and Data Warehouses?  
**A:** Start with a Data Lake.  As you query, you will discover cases where the data needs to be cleaned, combined, and made more accessible.  Create periodic Spark jobs to read these raw sources and write new "golden" tables that are cleaned and more easily queried.

## Next Steps
* Continue to the [Azure Data Lake Storage Gen2]($./07-Azure-Data-Lake-Gen2) lesson